# randrum

> Makes a random drum beat

In [ ]:
#| default_exp randrum

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
midi_note_to_drum_name = {
    35: "KickL",
    36: "KickR",
    37: "SnareRest",
    38: "Snare",
    41: "Tom4",
    42: "HihatClosed",
    45: "Tom3",
    46: "HihatOpen",
    47: "Tom2",
    48: "Tom1",
    49: "CrashL",
    51: "RideR",
    52: "China",
    53: "RideRBell",
    55: "RideLBell",
    57: "CrashR",
    59: "RideL",
}

note_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B', ]
note_names.extend(note_names)

midi_map2 = {i: note_names[i - 36] for i in range(36, 52)}

drum_name_to_note = {v: k for k, v in midi_note_to_drum_name.items()}

drum_names = list(drum_name_to_note.keys())
feet = ['KickL', 'KickR']
hands = ['SnareRest', 'Snare', 'Tom4', 'HihatClosed', 'Tom3', 'HihatOpen', 'Tom2', 'Tom1', 'CrashL', 'RideR', 'China', 'RideRBell', 'RideLBell', 'CrashR', 'RideL']


def drum_cleaner(drum_notes):
    # TODO
    # check that we do not have overlapping kicks
    # check that L and R kick alternate
    # make sure we do not have 3 hand drums playing at the same time
    cleaned_drum_notes = drum_notes
    return cleaned_drum_notes

In [ ]:
midi_map2

{36: 'C',
 37: 'C#',
 38: 'D',
 39: 'D#',
 40: 'E',
 41: 'F',
 42: 'F#',
 43: 'G',
 44: 'G#',
 45: 'A',
 46: 'A#',
 47: 'B',
 48: 'C',
 49: 'C#',
 50: 'D',
 51: 'D#'}

In [ ]:
#| export
import pretty_midi
import random
from freq_beat.manager import store_beat
from freq_beat.splitter import split_note

class Beat:
    def __init__(self) -> None:
        self.song = pretty_midi.PrettyMIDI()
        self.drums = pretty_midi.Instrument(program=0)
        self.possible_pitches = list(midi_map2.keys())
        self.beat_name = "rand-beat"
        self.notes = []
        
    def create_random_line(self, beat_length, song_length=10, play_prob=0.4):
        num_beats = int(song_length // beat_length)
        for i in range(num_beats):
            play = random.uniform(0, 1) < play_prob
            if play:
                pitch = random.choice(self.possible_pitches)
                start = i * beat_length
                end = start + beat_length
                note = pretty_midi.Note(start=start, end=end, velocity=80, pitch=pitch)
                split = random.uniform(0, 1) < 0.2
                if split:
                    notes = split_note(note, 4)
                    for piece in notes:
                        self.notes.append(piece)
                else:
                    self.notes.append(note)
                    
    def add_to_end(self, notes):
        current_end = max([note.end for note in self.notes])
        self.notes.extend(shift_notes(notes, current_end))
                    
    def create_random_song(self):
        # right now allowing three notes at the same time
        self.create_random_line(0.25, play_prob=.75)
        motif = Motif(shift_notes(self.notes[-10:]))
        motif_halftime = motif.concretize(2, {})
        motif_doubletime = motif.concretize(0.5, {})
        self.create_random_line(0.25)
        self.create_random_line(0.25)
        self.add_to_end(motif_halftime)
        self.add_to_end(motif_doubletime)

        for note in self.notes:
            self.drums.notes.append(note)
        self.song.instruments.append(self.drums)
        store_beat(self.song, self.beat_name)

In [ ]:
#| export
from freq_beat.motif import Motif
from freq_beat.manager import alter_note, shift_notes

class DrumBeat(Beat):
    def __init__(self) -> None:
        super().__init__()
        self.beat_name = "rand-drum-beat"
        
    def create_random_line(self, beat_length, drum_group, song_length=10, play_prob=0.4):
        num_beats = int(song_length // beat_length)
        for i in range(num_beats):
            play = random.uniform(0, 1) < play_prob
            if play:
                drum_to_play = random.choice(drum_group)
                pitch = drum_name_to_note[drum_to_play]
                start = i * beat_length
                end = start + beat_length
                note = pretty_midi.Note(start=start, end=end, velocity=80, pitch=pitch)
                split_rand = random.uniform(0, 1)
                if split_rand > 0.70:
                    if split_rand > 0.95:
                        pieces = split_note(note, 8)
                    elif split_rand > 0.85:
                        pieces = split_note(note, 4)
                    else:
                        pieces = split_note(note, 2)
                    for piece in pieces:
                        self.notes.append(piece)
                else:
                    self.notes.append(note)
                    
    def create_random_song(self):
        self.create_random_line(0.25, feet, play_prob=.75)
        self.create_random_line(0.25, hands)
        self.create_random_line(0.25, hands)
        for note in self.notes:
            self.drums.notes.append(note)
        self.song.instruments.append(self.drums)
        store_beat(self.song, self.beat_name)

In [ ]:
drum_beat = DrumBeat()
drum_beat.create_random_song()

Writing beat to beats/rand-drum-beat.mid.


In [ ]:
beat = Beat()
beat.create_random_song()

Writing beat to beats/rand-beat.mid.


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()